# Introduction to Memory Profiling

> Objectives:
> * Be introduced to memory profiling using different tools
> * Some small introduction to time profiling in IPython too


## ipython_memwatcher

Our recommended way to profile memory consumption for this tutorial will be [ipython_memwatcher](https://pypi.python.org/pypi/ipython_memwatcher):


In [ ]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [ ]:
import numpy as np

In [ ]:
na = np.ones(int(1e8)); nb = np.ones(int(1e8)); nc = np.ones(int(1e8))

In [ ]:
# Watch out the different RAM consumption mesurements
nd = na * nb + nc

In [ ]:
del na, nb, nc, nd

## Some mesasurements for basic containers

In [ ]:
# Let's create a big object
a = [i for i in range(1000*1000)]

In [ ]:
# Get some measurements from the last executed cell:
meas = mw.measurements
meas

In [ ]:
# MemWatcher.measurements is a named tuple.  We can easily get info out of it:
meas.memory_delta

In [ ]:
# This takes between 32 ~ 35 bytes per element:
meas.memory_delta * (2**20) / 1e6

In [ ]:
# What are these elements made of?
type(a[0])

In [ ]:
# How much memory take an int?
# Beware: the size below will depend on whether you are using a 32-bit or 64-bit Python
import sys
sys.getsizeof(a[0])

Ok.  On 64-bits, that means that the int object allocates 8 bytes for the integer value, and 16 bytes for other metadata (Python object).  But 24 is quite less than 32~35.  Where this overhead comes from?

Well, it turns out that the list structure needs additional pointers to reference the different components.

![List diagram](http://www.brpreiss.com/books/opus7/html/img579.gif)

## memory_profiler

[memory_profiler](https://pypi.python.org/pypi/memory_profiler) is a basic module for memory profiling that many others use (like the `ipython_memwatcher` above) and it interacts well with ipython, so it is worth to see how it works:

In [ ]:
%load_ext memory_profiler

In [ ]:
# Use %memit magic command exposed by memory_profiler
%memit b = [i for i in range(1000*1000)]

Please note that the `peak_memory` in this case is different than the `peaked_memory` reported by ipython_memwatcher package.

## %time and %timeit

In [ ]:
# IPython provides a magic command to see how much time a command takes to run
%time asum = sum(a)

Note that `%time` offers quite detailed statistics on the time spent.

Also, the time reported by MemoryWatcher has a typical overhead of 1~5 ms over the time reported by %time, so when the times to measure are about this order then it is better to rely on the %time (or %timeit below) values.  

In [ ]:
# We have another way to measure timings doing several loops and getting the mean
%timeit bsum = sum(a)
# However, one must notice that %timeit does not return the result of expressions

Interestingly, %timeit allows to retrieve the measured times in loops with the -o flag:

In [ ]:
t = %timeit -o sum(a)
print(t.all_runs)
print(t.best)

And one can specify the number of loops (-n) and the number of repetitions (-r):

In [ ]:
t = %timeit -r1 -n1 -o sum(a)
print(t.all_runs)
print(t.best)

### Exercise 1

Provided a dictionary like:

```
d = dict(("key: %i"%i, i*2) for i in a)
```

Try to guess how much RAM it uses using the techniques introduced above.

Why do you think it takes more space than a list?

*Hint*: Every entry in a dictionary has pointers to two objects: key and value. 

## Solution

In [ ]:
d = dict(("key: %i"%i, i*2) for i in a)

In [ ]:
# Compute the size of key + value
sys.getsizeof("key: 100000") + sys.getsizeof(1)

So, `sys.getsizeof()` is telling us that the keys + values take ~84 MB.  However, our `MemWatcher` instance is reporting just ~150 MB, so that means that the dict structure takes around ~66 MB.  The take away lesson is: do not assume that the consumption is just in the elements; the data container can also take quite a bit of memory!